In [1]:
import pandas as pd
import itertools
import numpy as np
import json
import os
import models
from handling_data import DataHandling
from trainer import Trainer
from analyzer import Analyzer

In [2]:
model_name= 'vqc'
version = 1

In [3]:
def load_json_file(model_name, version, submission_number):
    # Define the directory path
    path = f"./Submitted_Configurations/Version_{version}/{model_name}/{submission_number}.json"
    # Load the JSON file
    if not os.path.exists(path):
        raise FileNotFoundError(f"The file {path} does not exist.")
    with open(path, 'r') as file:
        data = json.load(file)
    
    path_trained = f"./Analyzed_Configurations/Version_{version}/{model_name}"
    if not os.path.exists(path_trained):
        os.makedirs(path_trained)
    # Write the data as a JSON file in the path_trained
    output_path = os.path.join(path_trained, f"{submission_number}.json")
    with open(output_path, 'w') as outfile:
        json.dump(data, outfile, indent=4)
    return data

def generate_combinations(param_dict):
    keys, values = zip(*param_dict.items())
    combinations = [dict(zip(keys, combo)) for combo in itertools.product(*values)]
    return combinations

def extract(config):
    # Extract parameters
    random_id = config["random_ids"]
    data_label = config["data_labels"]
    learning_rate = config["learning_rates"]
    num_qubits = config["num_qubits"]
    hidden_size = config["hidden_sizes"]
    ansatz = config["ansatz_types"]
    seq_length = config["sequence_lengths"]
    prediction_step = config["prediction_steps"]
    batch_size = config["batch_sizes"]
    if model_name == "vqc":
        model = models.VQC(num_qubits=num_qubits, seq_length=seq_length, ansatz=ansatz, data_label=data_label, random_id=random_id, evaluation=True)
    elif model_name == "qlstm_paper":
        model = models.QLSTM_Paper(num_qubits=num_qubits, ansatz=ansatz, data_label=data_label, random_id=random_id)
    elif model_name == "qlstm_linear_enhanced_paper":
        model = models.QLSTM_Linerar_Enhanced_Paper(num_qubits=num_qubits, hidden_size=hidden_size, ansatz=ansatz, data_label=data_label, random_id=random_id)
    elif model_name == "qrnn_paper":
        model = models.QRNN_Paper(num_qubits=num_qubits, num_qubits_hidden=hidden_size, seq_length=seq_length, ansatz=ansatz, data_label=data_label, random_id=random_id)
    elif model_name == "lstm":
        model = models.LSTM(hidden_size=hidden_size, ansatz=ansatz, data_label=data_label, random_id=random_id)
    elif model_name == "rnn":
        model = models.RNN(hidden_size=hidden_size, ansatz=ansatz, data_label=data_label, random_id=random_id)
    elif model_name == "mlp":
        model = models.MLP(seq_length=seq_length, ansatz=ansatz, data_label=data_label, random_id=random_id)
    data_handler = DataHandling(data_label=data_label, seq_length=seq_length, prediction_step=prediction_step)
    trainer = Trainer(model=model, random_id=random_id, learning_rate=learning_rate, batch_size=batch_size)
    analyzer = Analyzer(version=version, model=model, trainer=trainer, data_handler=data_handler)
    if analyzer.load_model():
        if not os.path.exists(analyzer.path + "/loss_metrics.csv"):
            return None
        loss_metrics = pd.read_csv(analyzer.path + "/loss_metrics.csv")
        training_info = pd.read_csv(analyzer.path + "/training_info.csv")
        mse_testing = loss_metrics["MSE Testing"][0]
        mse_validation = loss_metrics["MSE Validation"][0]
        mae_testing = loss_metrics["MAE Testing"][0]
        mae_validation = loss_metrics["MAE Validation"][0]
        corr_testing = loss_metrics["Correlation Testing"][0]
        corr_validation = loss_metrics["Correlation Validation"][0]
        num_parameters = analyzer.get_number_of_parameters()
        epochs_to_convergance = training_info["Epochs to Convergence"][0]
        total_training_time = training_info["Total Training Time"][0]
        training_loss_100_epochs = training_info["Training Loss after 100 epochs"][0]
        validation_loss_100_epochs = training_info["Validation Loss after 100 epochs"][0]
        testing_loss_100_epochs = training_info["Testing Loss after 100 epochs"][0]
        print(f"Model: {model_name}, Data: {data_label}, Random ID: {random_id}, Learning Rate: {learning_rate}, Num Qubits: {num_qubits}, Hidden Size: {hidden_size}, Ansatz: {ansatz}, Sequence Length: {seq_length}, Prediction Step: {prediction_step}, Batch Size: {batch_size}")
        tupel  = (version, model_name, ansatz, data_label, random_id, learning_rate, num_qubits, hidden_size, seq_length, prediction_step, batch_size, mse_testing, mse_validation, mae_testing, mae_validation, corr_testing, corr_validation, num_parameters, epochs_to_convergance, total_training_time, training_loss_100_epochs, validation_loss_100_epochs, testing_loss_100_epochs)
        return tupel

In [4]:
submission_numbers = [13, 14, 15, 16, 17, 18, 19, 20, 21]
file_path = f"./Results/ru_vqc_ansatz_optimization_results.csv"
tupel_list = []
for num in submission_numbers:
    configurations = load_json_file(model_name, version, num)
    combinations = generate_combinations(configurations)
    for combo in combinations:
        tupel = extract(combo)
        tupel_list.append(tupel)

df_new = pd.DataFrame(tupel_list, columns=["Version", "Model", "Ansatz", "Data", "Random ID", "Learning Rate", "Number Qubits", "Hidden Size", "Sequence Length", "Prediction Step", "Batch Size", "MSE Testing", "MSE Validation", "MAE Testing", "MAE Validation", "Correlation Testing", "Correlation Validation", "Num Parameters", "Epochs to Convergance", "Total Training Time", "Training Loss after 100 epochs", "Validation Loss after 100 epochs", "Testing Loss after 100 epochs"])
if os.path.exists(file_path):
    df_old = pd.read_csv(file_path)
    df = pd.concat([df_old, df_new], ignore_index=True)
else:
    df = df_new
df.to_csv(file_path, index=False)

Model: vqc, Data: mackey_1000, Random ID: 0, Learning Rate: 0.001, Num Qubits: 4, Hidden Size: None, Ansatz: ruexp_EXX_EXZ_CX_Y_EXX_CY, Sequence Length: 4, Prediction Step: 1, Batch Size: 64
Model: vqc, Data: mackey_1000, Random ID: 0, Learning Rate: 0.001, Num Qubits: 4, Hidden Size: None, Ansatz: ruexp_EXX_EXZ_CX_Y_EXX_CY, Sequence Length: 4, Prediction Step: 70, Batch Size: 64
Model: vqc, Data: mackey_1000, Random ID: 0, Learning Rate: 0.001, Num Qubits: 4, Hidden Size: None, Ansatz: ruexp_EXX_EXZ_CX_Y_EXX_CY, Sequence Length: 4, Prediction Step: 140, Batch Size: 64
Model: vqc, Data: mackey_1000, Random ID: 0, Learning Rate: 0.001, Num Qubits: 4, Hidden Size: None, Ansatz: ruexp_EXX_EXZ_CX_Y_EXX_CY, Sequence Length: 8, Prediction Step: 1, Batch Size: 64
Model: vqc, Data: mackey_1000, Random ID: 0, Learning Rate: 0.001, Num Qubits: 4, Hidden Size: None, Ansatz: ruexp_EXX_EXZ_CX_Y_EXX_CY, Sequence Length: 8, Prediction Step: 70, Batch Size: 64
Model: vqc, Data: mackey_1000, Random ID:

In [10]:
df = pd.read_csv("./Results/ru_vqc_ansatz_optimization_results.csv")
groupby_columns = ["Version", "Model", "Data", "Learning Rate", "Hidden Size", "Sequence Length", "Prediction Step", "Batch Size"]
grouped = df.groupby(groupby_columns)
output_list = []  # List to store dictionaries for JSON

with open('./Ru_vqc_ansatz/ten_best_models.json', 'w') as json_file:
    json_file.write("[\n")  # Start the JSON array
    first = True  # To manage commas between objects
    for param, indices in grouped.groups.items():
        group_df = df.loc[indices]
        group_df = group_df.nsmallest(10, 'MSE Validation')

        for num_qubits in [4, 6, 8]:
            df_filtered = group_df[group_df['Number Qubits'] == num_qubits]
            ansatz_list = df_filtered['Ansatz'].tolist()
            if len(ansatz_list) > 0:
                # Create dictionary for the current group
                data_dict = {
                    "version": [1],
                    "model_names": ["vqc"],
                    "random_ids": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                    "data_labels": [param[2]],  # 'Data' value from param tuple
                    "learning_rates": [0.001],
                    "num_qubits": [num_qubits],  # 'Number Qubits' value from param tuple
                    "hidden_sizes": [None],
                    "ansatz_types": ansatz_list,
                    "sequence_lengths": [int(param[5])],  # 'Sequence Length' value from param tuple
                    "prediction_steps": [int(param[6])],  # 'Prediction Step' value from param tuple
                    "batch_sizes": [int(param[7])]  # 'Batch Size' value from param tuple
                }
                if not first:
                    json_file.write(",\n")  # Separate dictionaries with commas
                json.dump(data_dict, json_file)  # Write dictionary as JSON object
                first = False  # After the first entry, we allow commas

    json_file.write("\n]")  # Close the JSON array


In [1]:
numbers = list(range(22, 92))
numbers

[22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91]